In [2]:
from upath import UPath

import torch
from dask.distributed import Client
from torch.optim import Adam
from tqdm.auto import tqdm

from uncle_val.datasets.dp1 import dp1_catalog_single_band
from uncle_val.learning.lsdb_dataset import LSDBIterableDataset
from uncle_val.learning.models import ConstantModel, LinearModel, MLPModel
from uncle_val.learning.losses import minus_ln_chi2_prob, kl_divergence_whiten
from uncle_val.learning.training import train_step

In [3]:
BAND = "r"

N_SRC = 10
BATCH_SIZE = 32

In [4]:
# DP1_ROOT = UPath("ssh://kmalanch@cmu.data.lsdb.io:/mnt/data/hats/catalogs/dp1")
DP1_ROOT = UPath("../../data/dp1")
assert DP1_ROOT.exists()

In [5]:
catalog = dp1_catalog_single_band(
    root=DP1_ROOT,
    band="r",
    obj="science",
    img="cal",
    phot="PSF",
    mode="forced",
).map_partitions(
    lambda df: df.query(
        "extendedness == 0",
    ).drop(
        columns=["id", "r_psfMag", "coord_ra", "coord_dec", "extendedness"],
    ),
)
catalog

,lc
npartitions=389,
"Order: 6, Pixel: 130","nested<x: [float], err: [float]>"
"Order: 8, Pixel: 2176",...
...,...
"Order: 9, Pixel: 2302101",...
"Order: 7, Pixel: 143884",...


In [6]:
model = LinearModel(d_input=2, d_output=1)
# model = torch.compile(model, mode='reduce-overhead')
model.train()

with Client(n_workers=8, memory_limit="8GB", threads_per_worker=1) as client:
    display(client)
    dataset = LSDBIterableDataset(
        catalog=catalog,
        columns=None,
        client=client,
        batch_lc=BATCH_SIZE,
        n_src=N_SRC,
        partitions_per_chunk=10,
        seed=0,
    )
    n_lc = 0
    optimizer = Adam(model.parameters(), lr=1e-3)
    for batch in tqdm(dataset):
        n_lc += len(batch)
        train_step(
            model=model,
            optimizer=optimizer,
            loss=minus_ln_chi2_prob,
            batch=batch,
        )
print(f"{n_lc = }")

model.eval()
torch.save(model, "model.pt")

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 59.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57514,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:57533,Total threads: 1
Dashboard: http://127.0.0.1:57537/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:57517,


/Users/hombit/.virtualenvs/uncle-val/lib/python3.13/site-packages/lsdb/catalog/dataset/healpix_dataset.py:740: RuntimeWarning: output of the function must be a DataFrame to generate an LSDB `Catalog`. `map_partitions` will return a dask object instead of a Catalog.
  warnings.warn(


0it [00:00, ?it/s]

n_lc = 161376


In [7]:
flux = 10 ** (-0.4 * (21 - 31.2))
err = 0.02 * flux
model(torch.tensor([flux, err]))

tensor([2.2703], grad_fn=<AsStridedBackward0>)